**1 - Configurando o Spark no Python**

Etapa responsável por realizar o Download e configuração do Apache Spark.

In [7]:
!ls 
!rm -rf *
#Instacao e configuracao do Java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Download do Apache Spark
!wget -q http://apache.osuosl.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
#Descompatacao do Apache Spark
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
#Instalacao do Findspark
!pip install -q findspark

spark-3.1.2-bin-hadoop3.2  spark-3.1.2-bin-hadoop3.2.tgz


**2 - Configurando o Ambiente**

Etapa responsável por configurar o ambiente do Spark

In [8]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

**3 - Configurando a sessão do Spark**

Etapa responsável por inicializar a sessão do Spark

In [9]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

**4 - Validando a Configuração e Versão do Spark**

Etapa de verificação 

In [10]:
import pyspark
print(pyspark.__version__)

3.1.2


**5 - Download dos Dados **
Etapa responsável por realizar o Download das informações de Gastos por meio de cartão de pagamento providas através da API: http://www.portaltransparencia.gov.br/api-de-dados/cartoes?pagina=1

Ou

CovidTracking através do: https://api.covidtracking.com/

Ou

Casas de Cambio:

https://www3.bcb.gov.br/vet/rest/v2/listaPontoCambio?cnpj=00000000



In [14]:
!curl -X GET --header 'Accept: application/json' 'https://www3.bcb.gov.br/vet/rest/v2/listaPontoCambio?cnpj=00000000' > consulta.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  150k    0  150k    0     0  65030      0 --:--:--  0:00:02 --:--:-- 65002


**6 - Carregando os dados no Spark**

Etapa responsável pela carga de dados no Spark.

In [15]:
#Instancia o contexto do Spark
sc = spark.sparkContext
#Realiza a leitura da nossa consulta realizada na API do Covid
path = "/content/consulta.json"
#Declara dataframe df com o conteudo da api que estava no JSON
df = spark.read.json(path)

**7 - Explorando os dados que foram obtidos**

Etapa de exploração dos dados.

In [16]:
#Imprime a estrutura de dados do nosso dataframe
df.printSchema()
#Imprime uma amosta de dados ( para especificar as linhas basta colocar a quantidade entre ())
df.show()

root
 |-- listaPontoCambio: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- cnpj: string (nullable = true)
 |    |    |-- endereco: string (nullable = true)
 |    |    |-- horaAbreDiaUtil: string (nullable = true)
 |    |    |-- horaAbreDomingo: string (nullable = true)
 |    |    |-- horaAbreFeriado: string (nullable = true)
 |    |    |-- horaAbreSabado: string (nullable = true)
 |    |    |-- horaFechaDiaUtil: string (nullable = true)
 |    |    |-- horaFechaDomingo: string (nullable = true)
 |    |    |-- horaFechaFeriado: string (nullable = true)
 |    |    |-- horaFechaSabado: string (nullable = true)
 |    |    |-- latitude: string (nullable = true)
 |    |    |-- longitude: string (nullable = true)
 |    |    |-- nome: string (nullable = true)
 |    |    |-- somenteClientes: boolean (nullable = true)
 |    |    |-- telefone1: string (nullable = true)
 |    |    |-- telefone2: string (nullable

**8 - Tratando e transformando os dados**

In [22]:
dfPrepared = df.selectExpr("explode(listaPontoCambio) as lista")
dfPrepared = dfPrepared.selectExpr("replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(lista.cidade,CHAR(192),'A'),CHAR(193),'A'),CHAR(194),'A'),CHAR(195),'A'),CHAR(196),'A'),CHAR(197),'A'),CHAR(224),'a'),CHAR(225),'a'),CHAR(226),'a'),CHAR(227),'a'),CHAR(228),'a'),CHAR(229),'a'),CHAR(200),'E'),CHAR(201),'E'),CHAR(202),'E'),CHAR(203),'E'),CHAR(232),'e'),CHAR(233),'e'),CHAR(234),'e'),CHAR(235),'e'),CHAR(204),'I'),CHAR(205),'I'),CHAR(206),'I'),CHAR(207),'I'),CHAR(236),'i'),CHAR(237),'i'),CHAR(238),'i'),CHAR(239),'i'),CHAR(210),'O'),CHAR(211),'O'),CHAR(212),'O'),CHAR(213),'O'),CHAR(214),'O'),CHAR(240),'o'),CHAR(242),'o'),CHAR(243),'o'),CHAR(244),'o'),CHAR(245),'o'),CHAR(246),'o'),CHAR(217),'U'),CHAR(218),'U'),CHAR(219),'U'),CHAR(220),'U'),CHAR(249),'u'),CHAR(250),'u'),CHAR(251),'u'),CHAR(252),'u'),CHAR(199),'C'),CHAR(231),'c') as cidade",
        "CAST(lista.cnpj as long) as cnpj",
        "replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(lista.endereco,CHAR(192),'A'),CHAR(193),'A'),CHAR(194),'A'),CHAR(195),'A'),CHAR(196),'A'),CHAR(197),'A'),CHAR(224),'a'),CHAR(225),'a'),CHAR(226),'a'),CHAR(227),'a'),CHAR(228),'a'),CHAR(229),'a'),CHAR(200),'E'),CHAR(201),'E'),CHAR(202),'E'),CHAR(203),'E'),CHAR(232),'e'),CHAR(233),'e'),CHAR(234),'e'),CHAR(235),'e'),CHAR(204),'I'),CHAR(205),'I'),CHAR(206),'I'),CHAR(207),'I'),CHAR(236),'i'),CHAR(237),'i'),CHAR(238),'i'),CHAR(239),'i'),CHAR(210),'O'),CHAR(211),'O'),CHAR(212),'O'),CHAR(213),'O'),CHAR(214),'O'),CHAR(240),'o'),CHAR(242),'o'),CHAR(243),'o'),CHAR(244),'o'),CHAR(245),'o'),CHAR(246),'o'),CHAR(217),'U'),CHAR(218),'U'),CHAR(219),'U'),CHAR(220),'U'),CHAR(249),'u'),CHAR(250),'u'),CHAR(251),'u'),CHAR(252),'u'),CHAR(199),'C'),CHAR(231),'c') as endereco",
        "lista.horaAbreDiaUtil",
        "lista.horaAbreDomingo",
        "lista.horaAbreFeriado",
        "lista.horaAbreSabado",
        "lista.horaFechaDiaUtil", 
        "lista.horaFechaDomingo",
        "lista.horaFechaFeriado",
        "lista.horaFechaSabado",
        "lista.latitude",
        "lista.longitude",
        "replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(lista.nome,CHAR(192),'A'),CHAR(193),'A'),CHAR(194),'A'),CHAR(195),'A'),CHAR(196),'A'),CHAR(197),'A'),CHAR(224),'a'),CHAR(225),'a'),CHAR(226),'a'),CHAR(227),'a'),CHAR(228),'a'),CHAR(229),'a'),CHAR(200),'E'),CHAR(201),'E'),CHAR(202),'E'),CHAR(203),'E'),CHAR(232),'e'),CHAR(233),'e'),CHAR(234),'e'),CHAR(235),'e'),CHAR(204),'I'),CHAR(205),'I'),CHAR(206),'I'),CHAR(207),'I'),CHAR(236),'i'),CHAR(237),'i'),CHAR(238),'i'),CHAR(239),'i'),CHAR(210),'O'),CHAR(211),'O'),CHAR(212),'O'),CHAR(213),'O'),CHAR(214),'O'),CHAR(240),'o'),CHAR(242),'o'),CHAR(243),'o'),CHAR(244),'o'),CHAR(245),'o'),CHAR(246),'o'),CHAR(217),'U'),CHAR(218),'U'),CHAR(219),'U'),CHAR(220),'U'),CHAR(249),'u'),CHAR(250),'u'),CHAR(251),'u'),CHAR(252),'u'),CHAR(199),'C'),CHAR(231),'c') as nome",
        "CAST(lista.somenteClientes as string) as somenteClientes",
        "lista.telefone1",
        "lista.telefone2",
        "lista.tipoAtendimento",
        "lista.tipoServico",
        "lista.uf",
        "CAST(0 as long) as cnpj_instituicao",
        "'' as enderecoEletronico_instituicao", 
        "'' as nome_instituicao")
dfPrepared.show()

+--------------+----+--------------------+---------------+---------------+---------------+--------------+----------------+----------------+----------------+---------------+----------+----------+--------------------+---------------+---------------+---------+---------------+-----------+---+----------------+------------------------------+----------------+
|        cidade|cnpj|            endereco|horaAbreDiaUtil|horaAbreDomingo|horaAbreFeriado|horaAbreSabado|horaFechaDiaUtil|horaFechaDomingo|horaFechaFeriado|horaFechaSabado|  latitude| longitude|                nome|somenteClientes|      telefone1|telefone2|tipoAtendimento|tipoServico| uf|cnpj_instituicao|enderecoEletronico_instituicao|nome_instituicao|
+--------------+----+--------------------+---------------+---------------+---------------+--------------+----------------+----------------+----------------+---------------+----------+----------+--------------------+---------------+---------------+---------+---------------+-----------+---+-

**9 - Analise das informações e resultados obtidos**

Etapa onde são realizadas as analises utilizando linguagem SQL para por exemplo descobrir quem mais gastou no ano vigente.

In [24]:
#Declara tabela temporaria no SparkSQL
dfPrepared.createOrReplaceTempView("dfPrepared")

In [25]:
df2 = spark.sql("select count(*),cidade from dfPrepared group by 2 order by 1 desc")
df2.show()

+--------+-------------------+
|count(1)|             cidade|
+--------+-------------------+
|      78|     Rio de Janeiro|
|      40|          Sao Paulo|
|      21|           Brasilia|
|       9|          Fortaleza|
|       7|           Campinas|
|       7|     Belo Horizonte|
|       7|           Salvador|
|       6|            Vitoria|
|       6|       Porto Alegre|
|       5|              Natal|
|       4|      Florianopolis|
|       4|Sao Jose dos Campos|
|       4|             Maceio|
|       3|           Curitiba|
|       3|             Santos|
|       3|              Bauru|
|       3|             Manaus|
|       3|        Joao Pessoa|
|       3|             Recife|
|       3|              Belem|
+--------+-------------------+
only showing top 20 rows

